### Si NO hay entorno Virtual
Primero , hay que ubicarse en la carpeta del programa del autoclicker en powershell usando 
```
cd "C:\path\to\HICKMICROTEST"

```
Luego, crear entorno virtual con 
```
python -m venv autopressvenv
```
seleccionar entorno en visual Studio
y correr el siguiente codigo:

In [ ]:
%pip install pywinauto pyautogui opencv-python numpy pillow pandas


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached pillow-12.0.0-cp313-cp313-win_amd64.whl.metadata (9.0 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel:


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Una vez creado el entorno virtual
En powershell de windows, hay que inicializar el entorno virtual con ejecutando el archivo `autopressvenv\Scripts\Activate.ps1`
una vez activado debe aparecer en el powershell `(autopressvenv) C:\Directorio\Actual`.

Luego, ejecutar archivo en powershell con el comando
 ```
 python auto_press_hikmicro.py
 ```
Con esto el autoclicker deberia funcionar en medida que no se cierre la ventana de powershell.


In [1]:
import cv2
import numpy as np

# ================= CONFIG =================

SCREENSHOT_PATH = "screenshots/lab_screen.jpeg"
TEMPLATE_PATH = "templates/restart_images.png"

MATCH_THRESHOLD = 0.65
SCALES = np.linspace(0.6, 1.4, 17)

# ==========================================

# Cargar imágenes
screen = cv2.imread(SCREENSHOT_PATH)
template = cv2.imread(TEMPLATE_PATH, cv2.IMREAD_GRAYSCALE)

if screen is None:
    raise FileNotFoundError("No se pudo cargar el pantallazo")

if template is None:
    raise FileNotFoundError("No se pudo cargar el template")

gray_screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)

# Preprocesado (igual que el script real)
gray_screen = cv2.GaussianBlur(gray_screen, (3, 3), 0)
template = cv2.GaussianBlur(template, (3, 3), 0)

edges_screen = cv2.Canny(gray_screen, 50, 150)
edges_tpl = cv2.Canny(template, 50, 150)

best_val = -1
best_loc = None
best_scale = None
best_size = None

for scale in SCALES:
    new_w = int(edges_tpl.shape[1] * scale)
    new_h = int(edges_tpl.shape[0] * scale)

    if new_w < 15 or new_h < 15:
        continue
    if new_w > edges_screen.shape[1] or new_h > edges_screen.shape[0]:
        continue

    tpl_resized = cv2.resize(edges_tpl, (new_w, new_h))
    res = cv2.matchTemplate(edges_screen, tpl_resized, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)

    if max_val > best_val:
        best_val = max_val
        best_loc = max_loc
        best_scale = scale
        best_size = (new_w, new_h)

print("\n===== RESULTADO =====")
print(f"Best score : {best_val:.3f}")
print(f"Best scale : {best_scale:.2f}")
print(f"Location   : {best_loc}")

if best_val >= MATCH_THRESHOLD:
    x, y = best_loc
    w, h = best_size

    cv2.rectangle(
        screen,
        (x, y),
        (x + w, y + h),
        (0, 255, 0),
        2
    )

    cv2.putText(
        screen,
        f"score={best_val:.2f}, scale={best_scale:.2f}",
        (x, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (0, 255, 0),
        2
    )

    print("MATCH ENCONTRADO ✔️")

else:
    print("NO SE ENCONTRO MATCH ❌")

cv2.imshow("Resultado deteccion", screen)
cv2.waitKey(0)
cv2.destroyAllWindows()



===== RESULTADO =====
Best score : 0.782
Best scale : 1.00
Location   : (620, 455)
MATCH ENCONTRADO ✔️
